<a href="https://colab.research.google.com/github/ryderwishart/nlp-model-experiments/blob/main/Greek_BERT_one_word_type.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# uncomment if you want to quickly delete unwanted files from previous model training
#import shutil
#shutil.rmtree('output')

Follow the general approach in this POS-tagging notebook: https://github.com/Kyubyong/nlp_made_easy/blob/master/Pos-tagging%20with%20Bert%20Fine-tuning.ipynb

In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import unicodedata
from google.colab import drive
import pandas as pd
import re
from transformers import BertTokenizer, BertForSequenceClassification
import torch
from transformers import TrainingArguments, Trainer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score
import numpy as np
from transformers import EarlyStoppingCallback
from collections import Counter
import json

drive.mount('/content/MyDrive')

Drive already mounted at /content/MyDrive; to attempt to forcibly remount, call drive.mount("/content/MyDrive", force_remount=True).


load in data and dictionaries

In [ ]:
data = '/content/MyDrive/MyDrive/Greek Bert Experiment/macula-greek.tsv'


In [ ]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [ ]:
data = pd.read_csv(data, sep='\t', keep_default_na=False, encoding='utf-8')
data.head()

,xml:id,ref,role,class,type,gloss,text,after,lemma,normalized,strong,morph,person,number,gender,case,tense,voice,mood,degree,domain,ln,frame,subjref,referent
0,n40001001001,MAT 1:1!1,,noun,common,[The] book,Βίβλος,,βίβλος,Βίβλος,976,N-NSF,,singular,feminine,nominative,,,,,033005,33.38,,,
1,n40001001002,MAT 1:1!2,,noun,common,of [the] genealogy,γενέσεως,,γένεσις,γενέσεως,1078,N-GSF,,singular,feminine,genitive,,,,,010002 033003,10.24 33.19,,,
2,n40001001003,MAT 1:1!3,,noun,proper,of Jesus,Ἰησοῦ,,Ἰησοῦς,Ἰησοῦ,2424,N-GSM,,singular,masculine,genitive,,,,,093001,93.169a,,,
3,n40001001004,MAT 1:1!4,,noun,proper,Christ,Χριστοῦ,,Χριστός,Χριστοῦ,5547,N-GSM,,singular,masculine,genitive,,,,,093001,93.387,,,
4,n40001001005,MAT 1:1!5,,noun,common,son,υἱοῦ,,υἱός,υἱοῦ,5207,N-GSM,,singular,masculine,genitive,,,,,010002,10.30,,,


In [ ]:
data = data[['text', 'type']]
data.head()

,text,type
0,Βίβλος,common
1,γενέσεως,common
2,Ἰησοῦ,proper
3,Χριστοῦ,proper
4,υἱοῦ,common


In [ ]:
type_int = {}

label = 0
types = data['type'].tolist()
for i in range(len(types)):
  if types[i] not in type_int:
    type_int[types[i]] = label
    types[i] = type_int[types[i]]
    label += 1
  else:
    types[i] = type_int[types[i]]

data = data.drop('type', axis=1)
data['class'] = types

data.head()

,text,class
0,Βίβλος,0
1,γενέσεως,0
2,Ἰησοῦ,1
3,Χριστοῦ,1
4,υἱοῦ,0


In [ ]:
type_int

{'common': 0,
 'proper': 1,
 '': 2,
 'personal': 3,
 'relative': 4,
 'demonstrative': 5,
 'interrogative': 6,
 'indefinite': 7,
 'possessive': 8,
 'adverbial': 9}

# BERT MODEL

In [ ]:
''' 
other models tested include: 
nlpaueb/bert-base-greek-uncased-v1, 
pranaydeeps/Ancient-Greek-BERT, 
bert-base-multilingual-cased, 
bert-base-multilingual-uncased
'''
pretrained = "pranaydeeps/Ancient-Greek-BERT"

In [ ]:
tokenizer = BertTokenizer.from_pretrained(pretrained)
model = BertForSequenceClassification.from_pretrained(pretrained,
                                                      num_labels = len(type_int))

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at pranaydeeps/Ancient-Greek-BERT and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
model = model.to('cuda')

In [ ]:
# process data
# RYDER: might need to add a for-loop and process each sentence in the input data
X = list(data['text'])
y = list(data['class'])
# 70/20/10 split for train, val, and test
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2) # 80/20
X_train, X_test, y_train, y_test = train_test_split(X_train, y_train, test_size=0.125) # 70/10
X_train_tokenized = tokenizer(X_train, padding=True, truncation=True, max_length=512)
X_val_tokenized = tokenizer(X_val, padding=True, truncation=True, max_length=512)

In [ ]:
X_train[:5]

['ἐστὶν', 'οἱ', 'σημεῖον', 'ἐκπέσητε', 'ἡ']

In [ ]:
print(f'{len(X_train)} items in the training set.')
print(f'{len(X_val)} items in the validation set.')
print(f'{len(X_test)} items in the test set.')

96445 items in the training set.
27556 items in the validation set.
13778 items in the test set.


In [ ]:
 # Create dataset
 class Dataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels=None):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        if self.labels:
            item["labels"] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.encodings["input_ids"])


In [ ]:
train_dataset = Dataset(X_train_tokenized, y_train)
val_dataset = Dataset(X_val_tokenized, y_val)

In [ ]:
def compute_metrics(p):
    pred, labels = p
    pred = np.argmax(pred, axis=1)

    accuracy = accuracy_score(y_true=labels, y_pred=pred)
    recall = recall_score(y_true=labels, y_pred=pred, average='weighted')
    precision = precision_score(y_true=labels, y_pred=pred, average='weighted')
    f1 = f1_score(y_true=labels, y_pred=pred, average='weighted')

    return {"accuracy": accuracy, "precision": precision, "recall": recall, "f1": f1}

In [ ]:
args = TrainingArguments(
    output_dir="/content/MyDrive/MyDrive/Greek Bert Experiment/output",
    evaluation_strategy="steps",
    eval_steps=500,
    per_device_train_batch_size=2048,
    per_device_eval_batch_size=2048,
    num_train_epochs=12,
    seed=0,
    load_best_model_at_end=True,
)
trainer = Trainer(
    model=model,
    args=args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics,
    #callbacks=[EarlyStoppingCallback(early_stopping_patience=3)],
)


In [ ]:
trainer.train()

Step,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
500,0.030700,0.048284,0.986609,0.986279,0.986609,0.986017


/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
stopruntime

#TEST

In [ ]:
X_test[:5]

In [ ]:
# true values
y_test[:5]

In [ ]:
X_test_tokenized = tokenizer(X_test, padding=True, truncation=True, max_length=512)
test_dataset = Dataset(X_test_tokenized)

In [ ]:
# CHANGE MODEL PATH TO BEST PERFORMING MODEL
model_path = "/content/MyDrive/MyDrive/Greek Bert Experiment/output/checkpoint-2000"
model = BertForSequenceClassification.from_pretrained(model_path, num_labels=len(type_int), ignore_mismatched_sizes=True)

In [ ]:
test_trainer = Trainer(model)

In [ ]:
raw_pred, _, _ = test_trainer.predict(test_dataset)

In [ ]:
y_pred = np.argmax(raw_pred, axis=1)

In [ ]:
predictions = y_pred.tolist()

In [ ]:
predictions[:5]

In [ ]:
inv_pos_int = {v: k for k, v in type_int.items()}

In [ ]:
type_true = y_test.copy()
type_predictions = predictions.copy()

In [ ]:
for i in range(len(type_true)):
  try:
    type_true[i] = inv_pos_int[type_true[i]]
  except:
    pass

for i in range(len(type_predictions)):
  type_predictions[i] = inv_pos_int[type_predictions[i]]

In [ ]:
print(type_true[:10])
print(type_predictions[:10])

In [ ]:
wrong = 0
total = 0

for i in range(len(type_true)):
  if type_true[i] != type_predictions[i]:
    wrong += 1
    total += 1
  else:
    total += 1

print(1-(wrong/total))

#Split Up Data into Groups

Analyze performance between content words (common nouns, proper nouns, adjectives, verbs, adverbs) and function words (i.e. conjunctions and prepositions)


In [ ]:
content = []
function = []


for i in range(len(pos_true)):
  if pos_true[i] in ('noun', 'adjective', 'verb'):
    content.append(tuple((X_test[i], pos_true[i], pos_predictions[i])))
  else:
    function.append(tuple((X_test[i], pos_true[i], pos_predictions[i])))

Analyze performance between words seen in the training dataset and those only seen in the test dataset

In [ ]:
in_train = []
out_train = []

for i in range(len(pos_true)):
  if (X_test[i] in X_test) or (X_test[i] in X_val):
    in_train.append(tuple((X_test[i], pos_true[i], pos_predictions[i])))
  else:
    out_train.append(tuple((X_test[i], pos_true[i], pos_predictions[i])))

In [ ]:
# adverbs make my brain hurt, so they're just going to be "function" words
pos_list = ['noun', 'adjective', 'verb', 'adverb', 
            'determiner', 'conjunction', 'preposition',
              'pronoun', 'particle', 'number', 'interjection']

Analyze performance between ambiguous (lemma occurs with multiple ln domains) and unambiguous (lemma occurs with single ln domain) words

In [ ]:
ambiguous_ln = []
ambiguous_words = []
unambiguous_ln = []

for i in range(len(X_test)):
  a = int_ln_dict[str(y_test[i])]
  b = int_ln_dict[str(predictions[i])]
  domains = lemma_ln_dict[X_test[i]]
  if len(domains) > 1:
    ambiguous_ln.append(tuple((a, b)))
    ambiguous_words.append(X_test[i])
  else:
    unambiguous_ln.append(tuple((a, b)))

# Functions

In [ ]:
def generate_incorrect_lists(set_list: list) -> list:
  incorrect_list = []

  for i in set_list:
    if i[1] != i[2]:
      incorrect_list.append(i)

  return incorrect_list

In [ ]:
def generate_incorrect_ambiguity_list(set_list: list, ambiguous_words: list) -> list:
  incorrect_list = []
  corresponding_domains = []

  for i in range(len(set_list)):
    if set_list[i][0] != set_list[i][1]:
      incorrect_list.append(set_list[i])
      corresponding_domains.append(lemma_ln_dict[ambiguous_words[i]])

  return incorrect_list, corresponding_domains

In [ ]:
def count_all_wrong(wrong_list: list, total: list):
  wrong_count = len(wrong_list)
  all_count = len(total)
  accuracy = wrong_count/all_count

  print(f'{wrong_count} domains incorrectly identified out of {all_count}.')
  print(f'{round(accuracy, 4)*100}% of domains incorrectly identified')

In [ ]:
def count_wrong_by_pos(pos_list: list, wrong_list: list) -> dict:
  key_list = pos_list
  wrong_pos_counts = {k:0 for k in key_list}
  
  for i in wrong_list:
    pos = i[0][1]
    wrong_pos_counts[pos] += i[1]
  
  return wrong_pos_counts

In [ ]:
def count_all_pos(pos_list: list, set_list: list) -> dict:
  key_list = pos_list
  all_pos_counts = {k:0 for k in key_list}

  try:
    for i in set_list:
      all_pos_counts[i] += 1
  except:
    for i in set_list:
      all_pos_counts[i[1]] += 1
    
  return all_pos_counts

In [ ]:
def create_pos_stats(wrong_pos_counts: dict, all_pos_counts: dict) -> list:   
  stats = []
  for i in wrong_pos_counts:
    if all_pos_counts[i] == 0:
          stats.append('{} out of {} {}s incorrect (0.0)'
              .format(wrong_pos_counts[i], all_pos_counts[i], i))
    else:
        stats.append('{} out of {} {}s incorrect ({})'
              .format(wrong_pos_counts[i], all_pos_counts[i], i, 
                      round((wrong_pos_counts[i]/all_pos_counts[i]), 4)))

  return stats

In [ ]:
def return_relevant_stats(pos_type: str, stats: list):
  pos_type = pos_type.lower()
  # if pos_list changes, adjust where list is split
  pos_dict = {
      'content': stats[:3],
      'function': stats[3:],
      'both': stats
  }

  if pos_type in pos_dict:
    for i in pos_dict[pos_type]:
      print(i)
  else:
    print('Please enter "content", "function", or "both" for pos_type.')

#Content word analysis

In [ ]:
incorrect_content = generate_incorrect_lists(content)
incorrect_content[:5]

In [ ]:
count_all_wrong(incorrect_content, content)

In [ ]:
cont_counts = Counter(incorrect_content).most_common()
cont_counts[:5]

In [ ]:
cont_wrong = count_wrong_by_pos(pos_list, cont_counts)
cont_all = count_all_pos(pos_list, pos_true)
cont_stats = create_pos_stats(cont_wrong, cont_all)
return_relevant_stats('content', cont_stats)

#Function Word Analysis

In [ ]:
incorrect_function = generate_incorrect_lists(function)
incorrect_function[:5]

In [ ]:
count_all_wrong(incorrect_function, function)

In [ ]:
func_counts = Counter(incorrect_function).most_common()
func_counts[:5]

In [ ]:
func_wrong = count_wrong_by_pos(pos_list, func_counts)
func_all = count_all_pos(pos_list, pos_true)
func_stats = create_pos_stats(func_wrong, func_all)
return_relevant_stats('function', func_stats)

# In Training Analysis

In [ ]:
in_train[:5]

In [ ]:
incorrect_in = generate_incorrect_lists(in_train)
count_all_wrong(incorrect_in, in_train)

In [ ]:
in_counts = Counter(incorrect_in).most_common()
in_counts[:5]

In [ ]:
in_wrong = count_wrong_by_pos(pos_list, in_counts)
in_all = count_all_pos(pos_list, in_train)
in_stats = create_pos_stats(in_wrong, in_all)
return_relevant_stats('both', in_stats)

# Out Of Training Analysis

In [ ]:
out_train[:5]

In [ ]:
incorrect_out = generate_incorrect_lists(out_train)
count_all_wrong(incorrect_out, out_train)

In [ ]:
out_counts = Counter(incorrect_out).most_common()
out_counts[:5]

In [ ]:
out_wrong = count_wrong_by_pos(pos_list, out_counts)
out_all = count_all_pos(pos_list, out_train)
out_stats = create_pos_stats(out_wrong, out_all)
return_relevant_stats('both', out_stats)

#Ambiguous Words

In [ ]:
ambiguous_ln[:5]

In [ ]:
incorrect_amb, amb_ln = generate_incorrect_ambiguity_list(ambiguous_ln, ambiguous_words)
count_all_wrong(incorrect_amb, ambiguous_ln)

In [ ]:
amb_counts = Counter(incorrect_amb).most_common()
amb_counts[:5]

In [ ]:
amb_wrong = count_wrong_ln_by_pos(pos_list, amb_counts)
amb_all = count_all_pos(pos_list, ambiguous_ln)
amb_stats = create_pos_stats(amb_wrong, amb_all)
return_relevant_stats('both', amb_stats)

Find the instances of these ln domains and see if the model is simply guessing the most common domain for these ambigous words.

In [ ]:
common_counts = 0
not_common_counts = 0
for i in amb_counts:
  j = incorrect_amb.index(i[0])
  current_dict = amb_ln[j]
  most_common = max(current_dict, key = current_dict.get)
  if most_common == i[0][1]:
    common_counts += i[1]
  else:
    not_common_counts += i[1]

print(f'model incorrectly guessed the most common domain {common_counts} times')
print(f'model incorrectly guessed something besides the most common domain {not_common_counts} times')

#Unambiguous Words

In [ ]:
unambiguous_ln[:5]

In [ ]:
incorrect_unamb = generate_incorrect_lists(unambiguous_ln)
count_all_wrong(incorrect_unamb, unambiguous_ln)

In [ ]:
unamb_counts = Counter(incorrect_unamb).most_common()
unamb_counts[:5]

In [ ]:
unamb_wrong = count_wrong_ln_by_pos(pos_list, unamb_counts)
unamb_all = count_all_pos(pos_list, unambiguous_ln)
unamb_stats = create_pos_stats(unamb_wrong, unamb_all)
return_relevant_stats('both', unamb_stats)

#Unfiltered Test Analsis

In [ ]:
mismatches = []

for i in range(len(pos_true)):
  if y_test[i] != predictions[i]:
    a = int_ln_dict[str(y_test[i])]
    b = int_ln_dict[str(predictions[i])]
    mismatches.append(tuple((a, b)))

mismatches[:5]

In [ ]:
all_wrong = generate_incorrect_lists(mismatches)
count_all_wrong(all_wrong, y_test)

In [ ]:
all_counts = Counter(mismatches).most_common()
all_counts[:5]

In [ ]:
wrong_counts = count_wrong_by_pos(pos_list, all_counts)
all_counts = count_all_pos(pos_list, y_test)
all_stats = create_pos_stats(wrong_counts, all_counts)
return_relevant_stats('both', all_stats)